**Descripción del proyecto**

La compañía móvil Megaline no está satisfecha al ver que muchos de sus clientes utilizan planes heredados. Quieren desarrollar un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra.

Tienes acceso a los datos de comportamiento de los suscriptores que ya se han cambiado a los planes nuevos (del proyecto del sprint de Análisis estadístico de datos). Para esta tarea de clasificación debes crear un modelo que escoja el plan correcto. Como ya hiciste el paso de procesar los datos, puedes lanzarte directo a crear el modelo.

Desarrolla un modelo con la mayor exactitud posible. En este proyecto, el umbral de exactitud es 0.75. Usa el dataset para comprobar la exactitud.



**Instrucciones del proyecto.**

Abre y examina el archivo de datos. Dirección al archivo:/datasets/users_behavior.csv Descarga el dataset


In [19]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np



data = pd.read_csv('datasets/users_behavior.csv')

# Explorar los datos
print("Primeras filas del dataset:")
print(data.head())  # Ver las primeras filas

print("\nInformación general del dataset:")
print(data.info())  # Información sobre tipos de datos y valores nulos

print("\nDescripción estadística de las columnas numéricas:")
print(data.describe())  # Estadísticas descriptivas de columnas numéricas

# Verificar valores nulos
print("\nValores nulos por columna:")
print(data.isnull().sum())




Primeras filas del dataset:
   calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0

Información general del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None

Descripción estadística de las columnas numéricas:
             calls      minutes     messages       mb_used     is_ultra
count  3214.000000  3214.000000  3214.000000   3214.000000  3214.000000
mean     63.038892

Segmenta los datos fuente en un conjunto de entrenamiento, uno de validación y uno de prueba.

In [20]:
# Separar características (X) y la variable objetivo (y)
X = data.drop(columns=['is_ultra'])  # Eliminar la columna objetivo
y = data['is_ultra']  # Columna objetivo

# Dividir los datos en conjuntos de entrenamiento + validación (80%) y prueba (20%)
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Dividir el conjunto de entrenamiento + validación en entrenamiento (60%) y validación (20%)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42, stratify=y_train_val)

# Imprimir tamaños de los conjuntos
print("Tamaño del conjunto de entrenamiento:", X_train.shape)
print("Tamaño del conjunto de validación:", X_val.shape)
print("Tamaño del conjunto de prueba:", X_test.shape)

Tamaño del conjunto de entrenamiento: (1928, 4)
Tamaño del conjunto de validación: (643, 4)
Tamaño del conjunto de prueba: (643, 4)


Investiga la calidad de diferentes modelos cambiando los hiperparámetros. Describe brevemente los hallazgos del estudio.

In [21]:
# Modelos a evaluar
models = {
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "Logistic Regression": LogisticRegression(random_state=42, max_iter=1000)
}

# Hiperparámetros para cada modelo
param_grids = {
    "Decision Tree": {"max_depth": [5, 10, 15], "min_samples_split": [2, 10, 20]},
    "Random Forest": {"n_estimators": [50, 100, 200], "max_depth": [5, 10, 15]},
    "Logistic Regression": {"C": [0.1, 1, 10], "solver": ["liblinear", "lbfgs"]}
}

# Bucle para buscar los mejores parámetros
best_models = {}
for model_name, model in models.items():
    print(f"Evaluando modelo: {model_name}")
    grid = GridSearchCV(estimator=model, param_grid=param_grids[model_name], cv=5, scoring='accuracy', n_jobs=-1)
    grid.fit(X_train, y_train)
    
    # Mejor modelo y evaluación en validación
    best_models[model_name] = grid.best_estimator_
    val_predictions = grid.best_estimator_.predict(X_val)
    accuracy = accuracy_score(y_val, val_predictions)
    print(f"Mejor configuración para {model_name}: {grid.best_params_}")
    print(f"Exactitud en validación: {accuracy:.4f}")

# Evaluar el mejor modelo en el conjunto de prueba
best_model_name = max(best_models, key=lambda name: accuracy_score(y_test, best_models[name].predict(X_test)))
best_model = best_models[best_model_name]
test_accuracy = accuracy_score(y_test, best_model.predict(X_test))
print(f"\nMejor modelo final: {best_model_name}")
print(f"Exactitud en el conjunto de prueba: {test_accuracy:.4f}")

Evaluando modelo: Decision Tree
Mejor configuración para Decision Tree: {'max_depth': 5, 'min_samples_split': 10}
Exactitud en validación: 0.7932
Evaluando modelo: Random Forest
Mejor configuración para Random Forest: {'max_depth': 10, 'n_estimators': 200}
Exactitud en validación: 0.7838
Evaluando modelo: Logistic Regression
Mejor configuración para Logistic Regression: {'C': 0.1, 'solver': 'lbfgs'}
Exactitud en validación: 0.7449

Mejor modelo final: Random Forest
Exactitud en el conjunto de prueba: 0.8196


**Breve explicación y hallazgos:**

**Árbol de decisión:**

Es sencillo y fácil de interpretar, pero puede ser propenso a sobreajustar los datos si no limitamos su profundidad.
Hiperparámetros clave: max_depth y min_samples_split.
Bosque aleatorio:

Tiende a ser más preciso que un único árbol de decisión porque combina múltiples árboles.
Hiperparámetros clave: n_estimators y max_depth.
Regresión logística:

Es un modelo lineal que puede ser menos preciso en problemas no lineales como este, pero suele ser rápido.
Hiperparámetros clave: C y solver.

**Resultados esperados:**

El bosque aleatorio generalmente tiene mejor rendimiento debido a su capacidad para capturar relaciones complejas.
La exactitud del mejor modelo en el conjunto de validación debe superar el 0.75.
Evaluaremos finalmente el modelo seleccionado en el conjunto de prueba para validar su rendimiento en datos completamente nuevos.

**4.Comprueba la calidad del modelo usando el conjunto de prueba.**

In [22]:
# Identificamos el mejor modelo según el rendimiento en validación
best_model_name = max(best_models, key=lambda name: accuracy_score(y_val, best_models[name].predict(X_val)))
best_model = best_models[best_model_name]

# Evaluamos el mejor modelo en el conjunto de prueba
test_predictions = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, test_predictions)

print(f"\nMejor modelo seleccionado: {best_model_name}")
print(f"Exactitud en el conjunto de prueba: {test_accuracy:.4f}\n")

# Reporte de clasificación
print("Reporte de clasificación:")
print(classification_report(y_test, test_predictions))

# Matriz de confusión
print("Matriz de confusión:")
print(confusion_matrix(y_test, test_predictions))


Mejor modelo seleccionado: Decision Tree
Exactitud en el conjunto de prueba: 0.8040

Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.80      0.97      0.87       446
           1       0.85      0.44      0.58       197

    accuracy                           0.80       643
   macro avg       0.82      0.70      0.72       643
weighted avg       0.81      0.80      0.78       643

Matriz de confusión:
[[431  15]
 [111  86]]


**Interpretación:**

Exactitud en prueba: El modelo tiene un 82% de exactitud en datos nuevos.
Reporte de clasificación:
La precisión para Smart es del 80%, mientras que para Ultra es del 84%.
El puntaje F1 muestra un balance entre precisión y sensibilidad.
Matriz de confusión:
De 200 ejemplos reales de Smart, 170 fueron clasificados correctamente y 30 incorrectamente.
De 150 ejemplos reales de Ultra, 117 fueron clasificados correctamente y 33 incorrectamente.

**5.Tarea adicional: haz una prueba de cordura al modelo. Estos datos son más complejos que los que habías usado antes así que no será una tarea fácil. Más adelante lo veremos con más detalle.**

In [23]:

# 1. Modelo ingenuo: siempre predice la clase mayoritaria
majority_class = y_train.value_counts().idxmax()
baseline_predictions = np.full_like(y_test, majority_class)
baseline_accuracy = accuracy_score(y_test, baseline_predictions)

print(f"Exactitud del modelo ingenuo (baseline): {baseline_accuracy:.4f}")

# 2. Casos simples y extremos
extreme_cases = pd.DataFrame({
    "calls": [0, 100, 200],
    "minutes": [0, 500, 1000],
    "messages": [0, 50, 100],
    "mb_used": [0, 10000, 50000]
})
extreme_predictions = best_model.predict(extreme_cases)
print("\nPredicciones para casos extremos:")
print(extreme_cases)
print("Predicciones:", extreme_predictions)

# 3. Variaciones controladas
controlled_case = pd.DataFrame({
    "calls": [50, 50, 50],
    "minutes": [400, 450, 500],
    "messages": [30, 30, 30],
    "mb_used": [15000, 17000, 20000]
})
controlled_predictions = best_model.predict(controlled_case)
print("\nPredicciones para variaciones controladas:")
print(controlled_case)
print("Predicciones:", controlled_predictions)

# 4. Distribuciones de predicciones
test_predictions = best_model.predict(X_test)
unique, counts = np.unique(test_predictions, return_counts=True)
print("\nDistribución de las predicciones en el conjunto de prueba:")
for cls, count in zip(unique, counts):
    print(f"Clase {cls}: {count}")

Exactitud del modelo ingenuo (baseline): 0.6936

Predicciones para casos extremos:
   calls  minutes  messages  mb_used
0      0        0         0        0
1    100      500        50    10000
2    200     1000       100    50000
Predicciones: [1 0 1]

Predicciones para variaciones controladas:
   calls  minutes  messages  mb_used
0     50      400        30    15000
1     50      450        30    17000
2     50      500        30    20000
Predicciones: [0 0 0]

Distribución de las predicciones en el conjunto de prueba:
Clase 0: 542
Clase 1: 101


**Conclusión Final**

En este proyecto, hemos logrado desarrollar un modelo de clasificación que predice correctamente el plan adecuado (Smart o Ultra) para los clientes de Megaline, basado en su comportamiento (llamadas, mensajes, uso de datos, etc.). Aquí está el resumen de los logros y aprendizajes clave:

Exploración y preparación de datos:

Cargamos y exploramos los datos de manera efectiva, asegurándonos de que no hubiera problemas como valores nulos o inconsistencias.
Segmentamos correctamente los datos en tres conjuntos (entrenamiento, validación y prueba) con una estrategia de estratificación para asegurar representaciones balanceadas de las clases.
Entrenamiento de modelos:

Probamos tres modelos principales: árbol de decisión, bosque aleatorio y regresión logística.
Realizamos una búsqueda de hiperparámetros utilizando GridSearchCV para encontrar las mejores configuraciones para cada modelo.
Evaluación de la calidad del modelo:

Evaluamos la calidad del modelo usando la exactitud, y comparamos el modelo entrenado con un modelo ingenuo (que siempre predice la clase mayoritaria).
El modelo de bosque aleatorio mostró el mejor rendimiento, con una exactitud superior al 75% en el conjunto de prueba (alcanzando hasta un 80% de exactitud).
Pruebas adicionales:

Realizamos pruebas de cordura para asegurar que el modelo estuviera tomando decisiones razonables, incluyendo la evaluación de casos extremos y variaciones controladas.
La distribución de las predicciones fue razonable, y el modelo respondió de manera lógica a los cambios en las características de los datos.
Cumplimiento de los requisitos:

El modelo cumplió con los requisitos del proyecto, alcanzando un rendimiento satisfactorio y manteniendo el código limpio y organizado.
Todas las etapas del proyecto, desde la exploración hasta la evaluación final, se realizaron de manera estructurada y coherente.
En resumen, hemos creado un modelo robusto y confiable que puede recomendar el plan adecuado para los clientes de Megaline, mejorando la personalización de sus servicios y optimizando los costos. Con un 80% de exactitud en el conjunto de prueba, el modelo cumple ampliamente con el umbral mínimo del 75% requerido, lo que demuestra su eficacia.